In [1]:
from azurewrapper.workspace import get_workspace

# subscription_id = "29d66431-a7ce-4709-93f7-3bdb01a243b3"
# resource_group = "ExperimentationJayke"
# workspace_name = "ExperimentationJayke"
subscription_id = "a00eaec6-b320-4e7c-ae61-60a30aec1cfc"
resource_group = "MachineLearning"
workspace_name = "RiverImageAnalysis"
tenant_id = "86f9fea7-9eb0-4325-8b58-7ed0db623956"

workspace = get_workspace(subscription_id, resource_group, workspace_name, tenant_id=tenant_id)

In [2]:
from azureml.core import Model

model = Model(workspace, "yolo_v4")

In [3]:
from azurewrapper.environment import get_environment

environment = get_environment(
    workspace,
    "tensorflow-yolo",
    pip_requirements="./examples/yolo/Tensorflow_YOLO/requirements.txt",
    docker_image="mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04"
)

No environment with that name found, creating new one


In [4]:
from azurewrapper.deploy import deploy
service = deploy(
    workspace,
    "testdeploy2",
    model,
    "score.py",
    "examples/yolo",
    environment=environment
)
service.wait_for_deployment(show_output = True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 0ea7be47d3e548c3bd0de28701c8bddf.azurecr.io
Logging into Docker registry 0ea7be47d3e548c3bd0de28701c8bddf.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 0ea7be47d3e548c3bd0de28701c8bddf.azurecr.io/azureml/azureml_2cac586b0b89d3965a18f9a23b0e8888
 ---> 8f9c70783517
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> db72f77902dc
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImEwMGVhZWM2LWIzMjAtNGU3Yy1hZTYxLTYwYTMwYWVjMWNmYyIsInJlc291cmNlR3JvdXBOYW1lIjoibWFjaGluZWxlYXJuaW5nIiwiYWNjb3VudE5hbWUiOiJyaXZlcmltYWdlYW5hbHlzaXMiLCJ3b3Jrc3BhY2VJZCI6IjBlYTdiZTQ3LWQzZTUtNDhjMy1iZDBkLWUyODcwMWM4YmRkZiJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in a5b87665acd9
 ---> 2d11bb4c36ad
Step 4/5 : RUN mv '/var/azureml-app/tmp10unxfys.py' /var/azureml-app/main.py
 ---> Running in

In [ ]:
service.get_logs()

In [12]:
service.reload()

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.


In [13]:
service.delete()

Container has been successfully cleaned up.
